In [2]:
import numpy as np 
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
#load the dataset
data = pd.read_csv('C:/Users/vidhya/Desktop/FRS/zomato.csv', encoding ='latin1')

In [3]:
data['City'].value_counts(dropna = False)

New Delhi         5473
Gurgaon           1118
Noida             1080
Faridabad          251
Ghaziabad           25
Ahmedabad           21
Bhubaneshwar        21
Lucknow             21
Guwahati            21
Amritsar            21
Ludhiana            20
Dubai               20
Valdosta            20
Indore              20
Des Moines          20
Kanpur              20
Bangalore           20
Columbus            20
Mysore              20
Varanasi            20
Cape Town           20
Nagpur              20
Sí£o Paulo          20
Albany              20
Kochi               20
Rio de Janeiro      20
Sioux City          20
Waterloo            20
Ankara              20
Coimbatore          20
                  ... 
Flaxton              1
Lorn                 1
Dicky Beach          1
Vernonia             1
Miller               1
Cochrane             1
Palm Cove            1
Princeton            1
Huskisson            1
Macedon              1
Mayfield             1
Phillip Island       1
Montville  

In [4]:
data_city =data.loc[data['City'] == 'Gurgaon']

In [5]:
data_new_delphi=data_city[['Restaurant Name','Cuisines','Locality','Aggregate rating']]

In [6]:
data_new_delphi.loc[data['Locality'] == 'Ardee City']

,Restaurant Name,Cuisines,Locality,Aggregate rating
1169,Red,"Japanese, Thai, Chinese",Ardee City,3.0
1170,Rocketchefs,"Pizza, Desserts",Ardee City,3.1
1171,The Fresh Chicken Store,"Raw Meats, Fast Food",Ardee City,2.9
1172,Achoos Food Corner,"South Indian, Chinese, Mughlai",Ardee City,0.0
1173,Aha Bites,"Fast Food, North Indian",Ardee City,0.0
1174,Ancient Spice,"North Indian, Chinese",Ardee City,0.0
1175,Chimney - The Takeaway,"North Indian, South Indian, Chinese",Ardee City,0.0
1176,Costa Coffee,Cafe,Ardee City,0.0
1177,Good to Go,Raw Meats,Ardee City,0.0
1178,Gopi Sweets & Caters,"Mithai, North Indian",Ardee City,0.0


In [9]:
data_sample=[]
def restaurant_recommend_func(location,title):   
    global data_sample       
    global cosine_sim
    global sim_scores
    global tfidf_matrix
    global corpus_index
    global feature
    global rest_indices
    global idx
    
    # When location comes from function ,our new data consist only location dataset
    data_sample = data_new_delphi.loc[data_new_delphi['Locality'] == location]  
    
    # index will be reset for cosine similarty index because Cosine similarty index has to be same value with result of tf-idf vectorize
    data_sample.reset_index(level=0, inplace=True) 
    
    #Feature Extraction
    data_sample['Split']="X"
    for i in range(0,data_sample.index[-1]):
        split_data=re.split(r'[,]', data_sample['Cuisines'][i])
        for k,l in enumerate(split_data):
            split_data[k]=(split_data[k].replace(" ", ""))
        split_data=' '.join(split_data[:])
        data_sample['Split'].iloc[i]=split_data
        
    #TF-IDF vectorizer
    #Extracting Stopword
    tfidf = TfidfVectorizer(stop_words='english')
#Replace NaN for empty string
    data_sample['Split'] = data_sample['Split'].fillna('')
#Applying TF-IDF Vectorizer
    tfidf_matrix = tfidf.fit_transform(data_sample['Split'])
    tfidf_matrix.shape
    
    # Using for see Cosine Similarty scores
    feature= tfidf.get_feature_names()
#Cosine Similarity
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
    
    # Column names are using for index
    corpus_index=[n for n in data_sample['Split']]
       
    #Construct a reverse map of indices    
    indices = pd.Series(data_sample.index, index=data_sample['Restaurant Name']).drop_duplicates() 
    
    #index of the restaurant matchs the cuisines
    idx = indices[title]
#Aggregate rating added with cosine score in sim_score list.
    sim_scores=[]
    for i,j in enumerate(cosine_sim[idx]):
        k=data_sample['Aggregate rating'].iloc[i]
        if j != 0 :
            sim_scores.append((i,j,k))
            
    #Sort the restaurant names based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: (x[1],x[2]) , reverse=True)
# 10 similar cuisines
    sim_scores = sim_scores[0:10]
    rest_indices = [i[0] for i in sim_scores] 
  
    data_x =data_sample[['Restaurant Name','Aggregate rating']].iloc[rest_indices]
    
    data_x['Cosine Similarity']=0
    for i,j in enumerate(sim_scores):
        data_x['Cosine Similarity'].iloc[i]=round(sim_scores[i][1],2)
   
    return data_x
# Top 10 similar restaurant with cuisine of 'Pizza Hut' restaurant in Connaught Place
#val=input(str)
restaurant_recommend_func('Ardee City','Red')


C:\Users\vidhya\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vidhya\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\vidhya\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Restaurant Name,Aggregate rating,Cosine Similarity
0,Red,3.0,1.00
5,Ancient Spice,0.0,0.28
12,Narayan Fast Food Home,0.0,0.28
14,OMG Tiffinz,0.0,0.28
15,Sardarji Chicken Point,0.0,0.28
6,Chimney - The Takeaway,0.0,0.18
10,Late Night Food,0.0,0.18
3,Achoos Food Corner,0.0,0.15
